In [132]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import contractions
from sklearn.model_selection import StratifiedShuffleSplit

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1


In [133]:
from time import time
import numpy as np
import pandas as pd
import tensorflow as tf

# Import Regex to clean up tweets
import re

import nltk, string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Get Reviews
import requests
import json

# Get Tweets
import httplib2
import requests
import urllib3

# TF IDF Imports


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from scipy.sparse import csc_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from joblib import dump, load

# RNN & LSTM Imports

from keras.losses import BinaryCrossentropy
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional, Masking, SimpleRNN
from keras.preprocessing import sequence
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.optimizers import Adam
from keras import regularizers

# from tensorflow.keras.preprocessing.text import Tokenizer
# from tensorflow.keras.preprocessing.sequence import pad_sequences
import random

In [134]:
df = pd.read_csv('./datasets/filtered_data.csv')
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,explanation edits made username hardcore metal...,0,0,0,0,0,0
1,000103f0d9cfb60f,daww matches background colour seemingly stuck...,0,0,0,0,0,0
2,000113f07ec002fd,hey man really trying edit war guy constantly ...,0,0,0,0,0,0
3,0001b41b1c6bb37e,cant make real suggestions improvement wondere...,0,0,0,0,0,0
4,0001d958c54c6e35,sir hero chance remember page thats,0,0,0,0,0,0


Nombre de mots max dans un commentaire

In [135]:
df['comment_text'].apply(lambda x:len(str(x).split())).max()

1001

Préparation des données

In [136]:
xtrain, xvalid, ytrain, yvalid = train_test_split(df.comment_text.values, df.toxic.values, 
                                                  stratify=df.toxic.values, 
                                                  random_state=42, 
                                                  test_size=0.2, shuffle=True)

xtrain.shape, xvalid.shape, ytrain.shape, yvalid.shape

((11993,), (2999,), (11993,), (2999,))

In [137]:
# Tokenisation
token = Tokenizer(num_words=None)
token.fit_on_texts(list(xtrain) + list(xvalid))
xtrain_seq = token.texts_to_sequences(xtrain)
xvalid_seq = token.texts_to_sequences(xvalid)

# Rembourrage des séquences
max_len = 1000
xtrain_pad = pad_sequences(xtrain_seq, maxlen=max_len)
xvalid_pad = pad_sequences(xvalid_seq, maxlen=max_len)

# Vérification des dimensions
print("Dimensions de xtrain_pad :", xtrain_pad.shape)
print("Dimensions de xvalid_pad :", xvalid_pad.shape)

word_index = token.word_index
print("Nombre de mots dans l'index :", len(word_index))

Dimensions de xtrain_pad : (11993, 1000)
Dimensions de xvalid_pad : (2999, 1000)
Nombre de mots dans l'index : 48991


In [145]:
Emmbedding(10000, 100, input_length=1000)

NameError: name 'Emmbedding' is not defined

In [149]:
import keras
keras.__version__

'3.0.0'

In [144]:
model = Sequential([Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len),
                    SimpleRNN(300)])

model.summary()

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_length': 1000}

In [142]:
model = Sequential()
model.add(Embedding(len(word_index) + 1,
                     300,
                     input_length=max_len))
model.add(SimpleRNN(300))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.summary()

ValueError: Unrecognized keyword arguments passed to Embedding: {'input_length': 1000}

In [ ]:
gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=1.0)
config = tf.compat.v1.ConfigProto(gpu_options=gpu_options)
session = tf.compat.v1.Session(config=config)

2024-02-26 17:25:13.852536: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-26 17:25:13.852633: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-26 17:25:13.852652: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-26 17:25:13.852859: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-02-26 17:25:13.852869: I tensorflow/core/common_runtime/gpu/gpu

Entrainement du modèle

In [ ]:
history = model.fit(xtrain_pad, ytrain, epochs=5, batch_size=64)

Epoch 1/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 25s 122ms/step - accuracy: 0.8709 - loss: 0.3462
Epoch 2/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 23s 122ms/step - accuracy: 0.9555 - loss: 0.1285
Epoch 3/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 119ms/step - accuracy: 0.9882 - loss: 0.0404
Epoch 4/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 119ms/step - accuracy: 0.9985 - loss: 0.0083
Epoch 5/5
188/188 ━━━━━━━━━━━━━━━━━━━━ 22s 118ms/step - accuracy: 0.9994 - loss: 0.0029


In [ ]:
# affichage de l'accuracy
training_accuracy = history.history['accuracy'][-1]
print("Training Accuracy:", training_accuracy)

Training Accuracy: 0.9996675252914429
